In [6]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.proxy import *
import pandas as pd 

no_of_reviews = 1000
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
driver = webdriver.Chrome()
# Append your app store urls here
url = "https://www.capterra.com/p/144037/Zoom-Video-Conferencing/reviews/"

driver.get(url)
# WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

a = 1
for i in range(20): #加載更多留言
    a+=50
    print(a)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="LoadableReviewSection"]/div/div[5]/div['+ str(a) +']/div/div/button')))
    target = driver.find_element_by_xpath('//*[@id="LoadableReviewSection"]/div/div[5]/div['+ str(a) +']/div/div/button')
    driver.execute_script("arguments[0].scrollIntoView();", target)
    driver.execute_script('window.scrollBy(0,-100)')
    try:
        driver.find_element_by_xpath('//*[@id="LoadableReviewSection"]/div/div[5]/div['+ str(a) +']/div/div/button').click()
    except:
        pass


page = driver.page_source

soup_expatistan = BeautifulSoup(page, "html.parser")

expatistan_table = soup_expatistan.find("h1", class_="nb-type-xl")

print("App name: ", expatistan_table.string)

expand_pages = soup_expatistan.findAll("div", class_="nb-p-xl")
counter = 1

list_of_comments = []
for expand_page in expand_pages:
    try:
        print("\n===========\n")
        print("review：" + str(counter))
        print("Title: ", str(expand_page.find("div", class_="nb-type-lg-bold").text))

        allopinion = expand_page.findAll("div", class_="nb-text-md nb-my-xl")
        opinion_store=[]
        for opinion in allopinion:
            opinion_store.append(str(opinion.text))
        print(opinion_store)

        Overall = ''
        Pros = ''
        Cons = ''
        Switch = ''
        Switch_reason = ''
        Alter = ''
        Alter_reason = ''
        for opinion in opinion_store:
            if 'Overall:' in opinion:
                Overall = opinion.replace('Overall:\xa0','')
            elif 'Pros:' in opinion:
                Pros = opinion.replace('Pros:\xa0','')
            elif 'Cons:' in opinion:
                Cons = opinion.replace('Cons:\xa0','')
            elif 'Switched From:' in opinion:
                Switch = opinion.replace('Switched From:\xa0','')
            elif 'Reasons for Switching to Zoom Meetings:' in opinion:
                Switch_reason = opinion.replace('Reasons for Switching to Zoom Meetings:\xa0','')
            elif 'Alternatives Considered:' in opinion:
                Alter = opinion.replace('Alternatives Considered:\xa0','')
            elif 'Reasons for Choosing Zoom Meetings:' in opinion:
                Alter_reason = opinion.replace('Reasons for Choosing Zoom Meetings:\xa0','')
                
        #寫入DataFrame
        ndf = pd.DataFrame([{'Review':str(counter),
                     'Title':str(expand_page.find("div", class_="nb-type-lg-bold").text),
                     'Overall':Overall,
                     'Pros':Pros,
                     'Cons':Cons,
                     'Switch':Switch,
                     'Switch_reason':Switch_reason,
                     'Alter':Alter,
                     'Alter_reason':Alter_reason}],
                    columns = ['Review', 'Title', 'Overall', 'Pros', 'Cons', 'Switch', 'Switch_reason', 'Alter', 'Alter_reason'])
        list_of_comments.append(ndf)
        counter+=1
    except:
        pass

51
101
151
201
251
301
351
401
451
501
551
601
651
701
751
801
851
901
951
1001
App name:  Zoom Meetings Reviews


review：1
Title:  “Most  simple, reliable and efficient meeting software ever. Experience meetings at its best! ”
['Overall:\xa0I would simply say that it has been the software we ever needed. The new additional exiting and helpful features have taken the experience to new heights . Its simply the best.', "Pros:\xa0The simplicity and usability of the software,  the fact that users don't have to necessarily signup and login in order to attend a meeting (which made over 90% of my novice users  especially in remote villages). The controls well designed and implemented to offer the easiest way possible for users to use them. The meeting experience it offer in spite of slow internet speeds. The wonderful features that like chats, polls, break out rooms, third party integrations had given me the best way to have meetings. Zoom is the best software option for online meetings and i

Title:  “Easy to use and fairly priced”
['Overall:\xa0Zoom meetings have been an excellent resource for our company since signing on. They made working from mush easier on our staff and provided a way to connect with students even though we could not be on campus due to the pandemic. ', 'Pros:\xa0The Zoom support team has been awesome from the beginning and continues to be. The management side of the software is fairly clean and straightforward enough that a non-IT professional could potentially manage it for a small company. Another thing that is great about Zoom is they are always working to improve upon the product and adding new features. ', 'Cons:\xa0From an administration side, there is a lack of granular control in some areas (especially with phones). While you can still limit user access to certain things, there are some settings that fall under an umbrella versus being by themself. ', 'Alternatives Considered:\xa0Webex', 'Reasons for Choosing Zoom Meetings:\xa0Pricing and usab



review：68
Title:  “Zoom Excellence”
["Overall:\xa0Zoom has been a rocky road but where it's currently at today I'm very pleased to make the most out of it, on top of all of that I can use it for business or just to keep in touch with relatives especially during this pandemic and it's made a huge difference than the average phone call.", 'Pros:\xa0Great easy to use very reliably, I remember when it first came out it was a little buggy. Since then a few years have passed and gotten more into the business use of it and it just is a game changer you can see the other person in high quality, share your screen, and even set annotation to point things out. amazing! On top of all of that when it comes to business calls are very easy to manage and my clients can access them effortlessly which makes a huge difference since a higher close rate can be credited to face-to-face calls.', "Cons:\xa0I'm usually very picky but when it comes down to this product I have 0 complaints, does as it promises

['Overall:\xa0Overall,  Zoom has provided that easy & seamless transition from 100% in-person work to now 100% virtual work in a matter of 2 days. Also, Zoom has been crucial to our "office bonding"  since it has now been approximately 5 months of both seeing these coworkers. ', "Pros:\xa0Back in March, like the rest of the country, I was sent to work from home due to coronavirus. Since Zoom was already integrated into our GSuite & used sporadically, it made it a seamless transition, most importantly our meetings that had already been scheduled prior to being sent home. Part of my job is recruitment & onboarding, and with Zoom, I have been able to successfully hire & onboard several new employees, specifically during this COVID time. When I have had to hire en mass, I like to utilize the breakout room feature when I have to virtually verify our employees' I-9 documents. The breakout rooms allow for there to be privacy when showing these documents on camera, reducing the stress & anxiet



review：132
Title:  “Best of the best, thank you for perfecting video chat”
["Overall:\xa0I originally started using Zoom with a client who had a client out of state. I was really impressed with the interface and brought it into my own team. It's allowed us to expand our customer base seamlessly out of town, and drastically cut down on unnecessary travel. ", 'Pros:\xa0I tested it side by side with competitors and it hands down had the best performance, the least glitches, and the easiest user interface. ', "Cons:\xa0Just the price-it's expensive. That said, it's worth it. I was stoked when my doctor started offering it!", 'Reasons for Choosing Zoom Meetings:\xa0Just not impressed with performance, and the side by side test CLEARLY showed that. ', 'Reasons for Switching to Zoom Meetings:\xa0Skype was glitchy and Google Hangouts had poor quality. ']


review：133
Title:  “Excellent meeting tool and also excellent for customer support!”
['Overall:\xa0My overall experience is excellent, ex

...



review：821
Title:  “COVID-19 Savior for so many!!”
['Overall:\xa0No problems at all with using  for larger groups! ', "Pros:\xa0It's super easy to learn how to navigate and use. Helped so many with having this app! I do recommend ", "Cons:\xa0The only problem I've had is when so many people are logged in at the same time it freezes a tad....", 'Alternatives Considered:\xa0Google Meet', "Reasons for Choosing Zoom Meetings:\xa0Zoom is just the Best all around! Those 2 options above don't cost a thing to use but Zoom allows way more people to Join our groups.ARC is decent as well but still don't compare to Zoom.  Zoom only freezes when there is around 50 people online in group and the others freeze not so good usually around 5-10. ", 'Reasons for Switching to Zoom Meetings:\xa0Gave Zoom a try and it works abit better for our larger types of groups. ']


review：822
Title:  “I've used Zoom for years but 2020 was definitely when it shined.”
['Overall:\xa0For regular chats and meetings I ha

['Overall:\xa0In terms of the main reason for using Zoom (video conferencing) - it just works great', 'Pros:\xa0The quality of video and audio is excellent as long as you have a decent network (also in terms of no syncing problems, no delays, etc.)', 'Cons:\xa0The settings and configurations are partly managed in the client app and partly in the web application - not convenient at all', 'Switched From:\xa0Google Meet', 'Reasons for Switching to Zoom Meetings:\xa0Zoom works much better. Google Meet had many video/audio sync issues, delays, hangs, etc. - especially if the network had occasional "hiccups"']


review：857
Title:  “Great software and easy to use”
['Overall:\xa0Great, easy to use. Everybody, even the olders or the non-frequent technology users get to understand how it works without explainations', 'Pros:\xa0It is very user-friendly, it has all the critical features and they are easy to use. It is professional and you rapidly get used to it.', 'Cons:\xa0Sometimes I have issue 


review：892
Title:  “Zoom Meetings & Chat - User Review”
['Overall:\xa0My overall experience with Zoom is good. There are certainly other video chat software available, however, I would choose Zoom every time. Although, it is a bit difficult to install, the security features during installment make me feel more at ease, during a call.', 'Pros:\xa0What I like most about Zoom Meetings & Chat is the overall quality of the video calls. The picture comes in clear and they even offer an "enhance face" feature, to smooth the look of user\'s faces. Compared to the video/picture quality of other live video software, Zoom takes the cake!', "Cons:\xa0The one thing is didn't like with Zoom is the loss/drop video calls. Its imporatnat with any video chat services, to have a strong WiFi signal or great data plan, that can support a robust call.", "Reasons for Switching to Zoom Meetings:\xa0To be honest, Skype was consistenly freezing and dropping calls...I just couldn't stand it much longer. Especia

["Overall:\xa0Zoom meetings have been most beneficial for meeting virtually with large (10+) groups as the other video conferencing software available to our company can only show 9 videos. We use Zoom to gather with 100+ volunteers on a monthly basis and sharing the link is simple and straightforward for guest who aren't regular users.", 'Pros:\xa0There are a lot of features allowing for broad functionality , such as screen sharing for collaboration, live chat and breakout rooms for brainstorming and questions, host permissions for large conference calls and reactions for quick engagement. ', 'Cons:\xa0I have found that setting up a meeting is clunky. In order to generate a link, the meeting needs to initially be set up in Zoom. Then it is transferred to the email platform of the user, where the invite needs to continue to be set up. With multiple users having access to an account, ']


review：922
Title:  “Fantastic, reliable and crucial these days.”
['Overall:\xa0Fantastic experience

["Overall:\xa0We love Zoom Meetings! I was using it for meetings before the start of the pandemic but, since March 2020, it has become invaluable. I use it for work as a freelance paralegal, it lets me be able to meet with attorneys, meet with clients, schedule depositions, etc. with ease. I also sell Tupperware on the side and since we can't party in person yet, I have hosted successful Zoom Tupperware parties! ", 'Pros:\xa0I like the ease of use. I set up meetings and depositions for attorneys that I freelance for and the setup and execution is so intuitive. ', "Cons:\xa0What I actually like least isn't the software's fault, too many people don't know how to use it effectively and insist on either not muting themselves when they are in a noisy environment or not knowing they have to unmute in order to talk. It would be nice if there was a way for the software to know when you are talking and mute/unmute automatically.", 'Alternatives Considered:\xa0Webex']


review：993
Title:  “Easie

In [7]:
comments = pd.concat(list_of_comments, ignore_index=True)
comments.to_csv('test.csv', encoding='utf_8_sig')